In [35]:
import numpy as np
import pandas as pd
import csv
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import pickle

In [ ]:
with open('/content/dailymonthly.csv', encoding='utf-8-sig') as file_obj:

    # Create reader object by passing the file
    # object to DictReader method
    reader_obj = csv.DictReader(file_obj)
    with open("/content/XGBoost_CMIP_SSP126_SPEI.csv", "w+") as f:
        # Define a csv writer object
        writer = csv.writer(f)
        writer.writerow(['Loc', 'mse', 'rmse', 'mae'])
        # Iterate over each row in the csv file
        # using reader object
        for row in reader_obj:
            data=row['daily']
            print(row)

            ssp = pd.read_csv('/content/SSP126/' + data + '.csv').drop(['Unnamed: 0', 'X', 'Year', 'Month'], axis=1)
            ssp = ssp.replace([np.inf, -np.inf], np.nan)

            X=ssp.iloc[5: ,0:4]
            Y=ssp.iloc[5: , 4:5]

            imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
            imputer.fit(Y)
            Y = imputer.transform(Y)
            X['Spei6']=Y
            Y=X.iloc[:,4]
            ya=Y.iloc[10:]

            X_test = np.zeros((910,10))
            for i in range(910):
              X_test[i,:] = Y[i:i+10]
            X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
            Y_test = np.array(ya.iloc[:])
            Y_test = Y_test.reshape(Y_test.shape[0],1,1)

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

            loaded_model = pickle.load(open('/content/model/'+data+'.h5', 'rb'))

            Y_pred = loaded_model.predict(X_test)

            mse = mean_squared_error(Y_test, Y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(Y_test, Y_pred)

            print("XGBoost model evaluation report:")
            print("----------------------------")
            print("Mean Squared Error:",mse)
            rmse = round((np.sqrt(mse)) , 5)
            print("Root Mean Squared Error:",rmse)
            print("Mean Absolute Error:",mae)
            print("\n \n")

            # Write the results to the file
            writer.writerow([data, mse, rmse, mae])